# Import Libraries

In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType, PeftModel
from datasets import load_dataset, Dataset
import pandas as pd
import torch
import re
import string
from sklearn.model_selection import StratifiedKFold
import numpy as np
from transformers_interpret import SequenceClassificationExplainer

## Step 1: Load your model + tokenizer

In [2]:
loca = "./model/best_model_f"
model = AutoModelForSequenceClassification.from_pretrained(loca)
tokenizer = AutoTokenizer.from_pretrained(loca)

## Step 2: Load training data (minority examples only)

In [3]:
df = pd.read_csv("data/minority_train.csv")
df = df[["text", "label"]].dropna()

## Step 2.5: Data Cleaning

In [4]:
def preprocess_text_data(
    df,
    text_column="text",
    min_words=3,
    expand_contractions=True,
    clean_text_flag=True,
    clean_garbage_flag=True,
    delete_short_flag=True,
    stratified_shuffle=True,   # NEW: enable/disable stratified shuffle
    label_column="label",
    n_splits=6,
    random_state=421
):
    contractions_dict = {
        "can't": "cannot", "won't": "will not", "i'm": "i am", "it's": "it is", "he's": "he is",
        "she's": "she is", "they're": "they are", "we're": "we are", "you're": "you are",
        "i've": "i have", "don't": "do not", "didn't": "did not", "doesn't": "does not",
        "isn't": "is not", "aren't": "are not", "wasn't": "was not", "weren't": "were not",
        "hasn't": "has not", "haven't": "have not", "hadn't": "had not", "shouldn't": "should not",
        "wouldn't": "would not", "couldn't": "could not", "mustn't": "must not", "let's": "let us",
        "that's": "that is", "what's": "what is", "there's": "there is", "who's": "who is",
        "where's": "where is", "how's": "how is", "here's": "here is", "i'll": "i will",
        "you'll": "you will", "he'll": "he will", "she'll": "she will", "we'll": "we will",
        "they'll": "they will", "i'd": "i would", "you'd": "you would", "he'd": "he would",
        "she'd": "she would", "we'd": "we would", "they'd": "they would", "n't": " not",
        "'re": " are", "'s": " is", "'d": " would", "'ll": " will", "'ve": " have", "'m": " am"
    }
    contractions_re = re.compile('(%s)' % '|'.join(map(re.escape, contractions_dict.keys())))

    def expand(text):
        return contractions_re.sub(lambda m: contractions_dict[m.group(0)], text)

    def clean(text):
        if expand_contractions:
            text = expand(text)
        text = text.lower()
        text = re.sub(r"\b(?:href|http|https|www)\b|\b(?:href|http|https|www)\S+", "", text)
        text = re.sub(r"@\w+", "", text)
        text = re.sub(r"\s+#", " #", text)
        text = re.sub(r"\d+", "", text)
        punct_to_remove = string.punctuation.replace("!", "").replace("?", "").replace("'", "")
        text = text.translate(str.maketrans("", "", punct_to_remove))
        text = re.sub(r"\s+", " ", text).strip()
        return text

    if clean_text_flag:
        df[text_column] = df[text_column].astype(str).apply(clean)

    if clean_garbage_flag:
        df = df.dropna()
        df[text_column + '_norm'] = df[text_column].str.lower().str.strip()
        df = df.drop_duplicates(subset=[text_column + '_norm'])
        df = df.drop(columns=[text_column + '_norm'])
        df = df[df[text_column].str.strip().astype(bool)]

    if delete_short_flag:
        short_rows = df[df[text_column].apply(lambda x: len(str(x).split()) < min_words)]
        if not short_rows.empty:
            print(f"Short sentences (less than {min_words} words):")
            print(short_rows[[text_column]] if label_column not in df.columns else short_rows[[text_column, label_column]])
            confirm = input(f"\nDelete these {len(short_rows)} rows? (yes/no): ").strip().lower()
            if confirm == "yes":
                df = df.drop(short_rows.index).reset_index(drop=True)
                print(f"Deleted {len(short_rows)} short sentences.")
            else:
                print("No rows deleted.")

    # --- Stratified Shuffle (from your code) ---
    if stratified_shuffle:
        from sklearn.model_selection import StratifiedKFold
        import numpy as np
        skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
        indices = []
        for _, idx in skf.split(df[text_column], df[label_column]):
            idx = np.array(idx)
            np.random.shuffle(idx)  # Shuffle indices within each block
            indices.extend(idx)
        df = df.iloc[indices].reset_index(drop=True)

    return df

In [5]:
df = preprocess_text_data(df, text_column="text")

## Step 2.6: Label Mapping and Tokenization

In [6]:
label_list = sorted(df['label'].unique())
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for i, label in enumerate(label_list)}

df['label'] = df['label'].map(label2id)

dataset = Dataset.from_pandas(df)

def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding=True, max_length=128)

tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/493 [00:00<?, ? examples/s]

## Step 3: LoRA config

In [7]:
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none"
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 594,438 || all params: 185,021,196 || trainable%: 0.3213



## Step 4: Training args

In [8]:
training_args = TrainingArguments(
    output_dir="lora_checkpoints/minority_patch",
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 4,
    num_train_epochs=8,
    learning_rate=3e-5,
    logging_dir="./logs",
    logging_steps=20,
    save_strategy="no",
    fp16=True,
    report_to="none",
    label_smoothing_factor = 0.0,
    label_names=["label"],  # Add this line
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

C:\Users\SunAda\AppData\Local\Temp\ipykernel_27608\299073688.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


## Step 5: Train!


In [9]:
trainer.train()

Step,Training Loss
20,2.451500
40,2.022300
60,2.078400
80,1.742900
100,1.750600
120,1.595900
140,1.364600
160,1.253900
180,1.219600
200,1.019500


TrainOutput(global_step=248, training_loss=1.5346246457869006, metrics={'train_runtime': 63.2858, 'train_samples_per_second': 62.32, 'train_steps_per_second': 3.919, 'total_flos': 38778112082880.0, 'train_loss': 1.5346246457869006, 'epoch': 8.0})

## Step 6: Save LoRA adapter only

In [10]:
model.save_pretrained("lora_checkpoints/minority_patch")
tokenizer.save_pretrained("lora_checkpoints/minority_patch")

('lora_checkpoints/minority_patch\\tokenizer_config.json',
 'lora_checkpoints/minority_patch\\special_tokens_map.json',
 'lora_checkpoints/minority_patch\\tokenizer.json')

## Step 7: Sanity Check

In [11]:
# Load base
base_model = AutoModelForSequenceClassification.from_pretrained("model/best_model_f")

# Apply LoRA adapter
model = PeftModel.from_pretrained(base_model, 
                                  "lora_checkpoints/minority_patch", 
                                  label2id=label2id,
                                    id2label=id2label)
model.print_trainable_parameters()
# Sanity check: are LoRA adapters active?
print("Active adapters:", model.peft_config)

trainable params: 4,614 || all params: 185,021,196 || trainable%: 0.0025
Active adapters: {'default': LoraConfig(task_type='SEQ_CLS', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='./model/best_model_f', revision=None, inference_mode=True, r=16, target_modules={'query_proj', 'value_proj'}, exclude_modules=None, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=['classifier', 'score', 'classifier', 'score'], init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, use_qalora=False, qalora_group_size=16, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)}


# Testing 

In [12]:
def predict_emotion(texts):
    model.eval()
    predictions = []
    for text in texts:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(model.device)
        with torch.no_grad():
            logits = model(**inputs).logits
            probs = torch.nn.functional.softmax(logits, dim=1)
            label_id = torch.argmax(probs, dim=1).item()
            confidence = probs[0][label_id].item()
        predictions.append((text, label_id, confidence))
    return predictions

In [13]:
minority_test_samples = [
    "He smiled as he read the unexpected letter.",        # joy
    "Her hands shook as she heard the strange noise.",    # fear
    "He opened the door and gasped.",                          # surprise     
    "The envelope contained a ticket to a place I had never heard of",# surprise
    
]

results = predict_emotion(minority_test_samples)
for text, label_id, conf in results:
    label_id = id2label[label_id]
    print(f"{text} → {label_id} (conf: {conf:.2f})")

He smiled as he read the unexpected letter. → surprise (conf: 0.56)
Her hands shook as she heard the strange noise. → surprise (conf: 0.53)
He opened the door and gasped. → fear (conf: 0.52)
The envelope contained a ticket to a place I had never heard of → joy (conf: 0.46)


In [14]:
explainer = SequenceClassificationExplainer(
    model=model,
    tokenizer=tokenizer
)

word_attributions = explainer("He opened the door and gasped.")
explainer.visualize()  # or print(word_attributions)

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,fear (0.53),fear,-0.22,[CLS] ▁He ▁opened ▁the ▁door ▁and ▁gasped ▁. [SEP]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,fear (0.53),fear,-0.22,[CLS] ▁He ▁opened ▁the ▁door ▁and ▁gasped ▁. [SEP]


# Continue Training the Lora

## Step 9: Train Lora further

In [15]:
# Load CSV and convert to HuggingFace Dataset
df = pd.read_csv("./data/borderline_signal.csv")

# Optional: drop duplicates or near-duplicates
df = preprocess_text_data(df, text_column="text")

# Map label strings to ids
df["label_id"] = df["label"].map(label2id)

# Convert to HuggingFace Dataset
dataset = Dataset.from_pandas(df[["text", "label_id"]])


## Step 10: Tokenize new dataset

In [16]:
tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset = tokenized_dataset.rename_column("label_id", "label")
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/175 [00:00<?, ? examples/s]

## Step 11: Define Trainer

In [17]:
training_args = TrainingArguments(
    output_dir="./lora_checkpoints/minority_patch",
    per_device_train_batch_size=4,         # smaller batch → more granular LoRA updates
    gradient_accumulation_steps=2,         # effective batch size = 8
    num_train_epochs=6,                    # memorize-level training
    learning_rate=5e-5,                    # much better for LoRA tuning
    weight_decay=0.0,                      # no need — your examples are high-quality
    warmup_ratio=0.1,                      # optional, helps smooth out first few steps
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=False,
    remove_unused_columns=False,
    overwrite_output_dir=True,
    run_name="lora-phase2-borderline"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

C:\Users\SunAda\AppData\Local\Temp\ipykernel_27608\3823117642.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [18]:
trainer.train()

Step,Training Loss
10,2.001400
20,1.909000
30,1.789700
40,1.813500
50,1.839400
60,1.982300
70,1.656100
80,1.793600
90,1.845000
100,1.738500


TrainOutput(global_step=132, training_loss=1.8274004784497349, metrics={'train_runtime': 10.378, 'train_samples_per_second': 101.175, 'train_steps_per_second': 12.719, 'total_flos': 8150358510000.0, 'train_loss': 1.8274004784497349, 'epoch': 6.0})

# Testing

In [19]:
minority_test_samples = [
    "He smiled as he read the unexpected letter.",        # joy
    "Her hands shook as she heard the strange noise.",    # fear
    "He opened the door and gasped.",                              # surprise
    "The letter made her tear up with joy.",                       # joy (control)
    "I felt a deep hole in my chest all day",
    "He smiled as he read the unexpected letter.",        # joy
    "Her hands shook as she heard the strange noise.",    # fear
    "He opened the door and gasped.",                          # surprise     
    "The envelope contained a ticket to a place I had never heard of",# surprise
]

results = predict_emotion(minority_test_samples)
for text, label_id, conf in results:
    label_id = id2label[label_id]
    print(f"{text} → {label_id} (conf: {conf:.2f})")

He smiled as he read the unexpected letter. → surprise (conf: 0.52)
Her hands shook as she heard the strange noise. → surprise (conf: 0.64)
He opened the door and gasped. → surprise (conf: 0.54)
The letter made her tear up with joy. → joy (conf: 0.89)
I felt a deep hole in my chest all day → sadness (conf: 0.51)
He smiled as he read the unexpected letter. → surprise (conf: 0.52)
Her hands shook as she heard the strange noise. → surprise (conf: 0.64)
He opened the door and gasped. → surprise (conf: 0.54)
The envelope contained a ticket to a place I had never heard of → joy (conf: 0.47)


In [20]:
from transformers_interpret import SequenceClassificationExplainer

explainer = SequenceClassificationExplainer(
    model=model,
    tokenizer=tokenizer
)

word_attributions = explainer("I felt a deep hole in my chest all day")
explainer.visualize()  # or print(word_attributions)

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
4,sadness (0.51),sadness,2.47,[CLS] ▁I ▁felt ▁a ▁deep ▁hole ▁in ▁my ▁chest ▁all ▁day [SEP]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
4,sadness (0.51),sadness,2.47,[CLS] ▁I ▁felt ▁a ▁deep ▁hole ▁in ▁my ▁chest ▁all ▁day [SEP]


# Saving

In [21]:
model.save_pretrained("lora_checkpoints/minority_patch")
tokenizer.save_pretrained("lora_checkpoints/minority_patch")

('lora_checkpoints/minority_patch\\tokenizer_config.json',
 'lora_checkpoints/minority_patch\\special_tokens_map.json',
 'lora_checkpoints/minority_patch\\tokenizer.json')

In [22]:
base_model = AutoModelForSequenceClassification.from_pretrained("model/best_model_f",
                                                                label2id=label2id,
                                                                id2label=id2label)
# Apply LoRA adapter
model = PeftModel.from_pretrained(base_model, 
                                  "lora_checkpoints/minority_patch", 
                                  label2id=label2id,
                                    id2label=id2label)
model.print_trainable_parameters()
# Sanity check: are LoRA adapters active?
print("Active adapters:", model.peft_config)

trainable params: 4,614 || all params: 185,021,196 || trainable%: 0.0025
Active adapters: {'default': LoraConfig(task_type='SEQ_CLS', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='./model/best_model_f', revision=None, inference_mode=True, r=16, target_modules={'query_proj', 'value_proj'}, exclude_modules=None, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=['classifier', 'score', 'classifier', 'score', 'classifier', 'score'], init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, use_qalora=False, qalora_group_size=16, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)}


# Continue Training Lora

## Step 12: Load and Map

In [23]:
df = pd.read_csv("./data/logic_inversion.csv")
df = preprocess_text_data(df, text_column="text")

df["label_id"] = df["label"].map(label2id)

dataset = Dataset.from_pandas(df[["text", "label_id"]])

## Step 13: Tokenize

In [24]:
tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset = tokenized_dataset.rename_column("label_id", "label")
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/305 [00:00<?, ? examples/s]

## Step 14: Training Args

In [25]:
training_args = TrainingArguments(
    output_dir="./lora_checkpoints/logic_patch",
    per_device_train_batch_size=4,         # smaller batch → more granular LoRA updates
    gradient_accumulation_steps=2,         # effective batch size = 8
    num_train_epochs=6,                    # memorize-level training
    learning_rate=5e-5,                    # much better for LoRA tuning
    weight_decay=0.0,                      # no need — your examples are high-quality
    warmup_ratio=0.1,                      # optional, helps smooth out first few steps
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=False,
    remove_unused_columns=False,
    overwrite_output_dir=True,
    run_name="lora-phase3-inversion"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

C:\Users\SunAda\AppData\Local\Temp\ipykernel_27608\1920385646.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


## Step 15: Train

In [26]:
trainer.train()

Step,Training Loss
10,1.499800
20,1.317700
30,1.489200
40,1.334600
50,1.424900
60,1.490200
70,1.585700
80,1.279300
90,1.344300
100,1.567600


TrainOutput(global_step=234, training_loss=1.3985956102354913, metrics={'train_runtime': 17.2763, 'train_samples_per_second': 105.926, 'train_steps_per_second': 13.545, 'total_flos': 18939880728000.0, 'train_loss': 1.3985956102354913, 'epoch': 6.0})

# Testing

In [27]:
## Step 6: Save LoRA adapter only
model.save_pretrained("lora_checkpoints/logic_patch")
tokenizer.save_pretrained("lora_checkpoints/logic_patch")
# Load base
base_model = AutoModelForSequenceClassification.from_pretrained("model/best_model_f")

# Apply LoRA adapter
model = PeftModel.from_pretrained(base_model, 
                                  "lora_checkpoints/logic_patch", 
                                  label2id=label2id,
                                    id2label=id2label)
model.print_trainable_parameters()
# Sanity check: are LoRA adapters active?
print("Active adapters:", model.peft_config)

trainable params: 4,614 || all params: 185,021,196 || trainable%: 0.0025
Active adapters: {'default': LoraConfig(task_type='SEQ_CLS', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='./model/best_model_f', revision=None, inference_mode=True, r=16, target_modules={'query_proj', 'value_proj'}, exclude_modules=None, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=['classifier', 'score', 'classifier', 'score', 'classifier', 'score', 'classifier', 'score'], init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, use_qalora=False, qalora_group_size=16, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)}


In [28]:
eval_challenging_samples = [
    #anger
    "I clenched my jaw as they congratulated the thief.",
    "He smiled, and I wanted to scream.",
    "Each word she said felt like another slap.",
    "I broke the plate—not by accident.",
    "They praised him again, and I walked out.",
    #fear
    "My hands shook as the door creaked open.",
    "I pretended not to hear the footsteps behind me.",
    "The sudden silence in the woods felt too loud.",
    "Every shadow on the wall made my breath catch.",
    "He wasn’t supposed to be there, but he was.",
    #joy
    "I couldn't stop laughing, even with tears in my eyes.",
    "The music played and my heart danced.",
    "He brought coffee, and it was exactly how I like it.",
    "She said yes, and the world slowed down for a second.",
    "I watched the sunrise, completely at peace.",
    #love
    "She always remembers the little things I forget.",
    "Even in silence, we said everything.",
    "He held my hand like it was the only thing that mattered.",
    "She stayed up just to hear how my day went.",
    "I made dinner the way his mom used to.",
    #sadness
    "The laughter felt distant, like it wasn't mine.",
    "I stood in the same room, but everything had changed.",
    "Even his favorite song felt hollow today.",
    "She left the light on, but no one came back.",
    "I wanted to smile, but it just wouldn’t come.",
    #surprise
    "The box had my name, but I never told anyone.",
    "She opened the door, and there he was.",
    "He remembered the anniversary I forgot.",
    "That call at midnight changed everything.",
    "When I turned around, the crowd was cheering.",
   
]

results = predict_emotion(eval_challenging_samples)
for text, label_id, conf in results:
    label_id = id2label[label_id]
    print(f"{text} → {label_id} (conf: {conf:.2f})")
    print()

I clenched my jaw as they congratulated the thief. → anger (conf: 0.63)

He smiled, and I wanted to scream. → anger (conf: 0.78)

Each word she said felt like another slap. → anger (conf: 0.79)

I broke the plate—not by accident. → fear (conf: 0.47)

They praised him again, and I walked out. → anger (conf: 0.79)

My hands shook as the door creaked open. → fear (conf: 0.84)

I pretended not to hear the footsteps behind me. → fear (conf: 0.86)

The sudden silence in the woods felt too loud. → fear (conf: 0.50)

Every shadow on the wall made my breath catch. → fear (conf: 0.80)

He wasn’t supposed to be there, but he was. → fear (conf: 0.50)

I couldn't stop laughing, even with tears in my eyes. → surprise (conf: 0.84)

The music played and my heart danced. → surprise (conf: 0.81)

He brought coffee, and it was exactly how I like it. → love (conf: 0.74)

She said yes, and the world slowed down for a second. → surprise (conf: 0.83)

I watched the sunrise, completely at peace. → joy (conf: 

In [ ]:
explainer = SequenceClassificationExplainer(
    model=model,
    tokenizer=tokenizer
)

word_attributions = explainer("He remembered the anniversary I forgot.")
explainer.visualize()  # or print(word_attributions)

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
5,surprise (0.35),surprise,0.17,[CLS] ▁He ▁remembered ▁the ▁anniversary ▁I ▁forgot ▁. [SEP]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
5,surprise (0.35),surprise,0.17,[CLS] ▁He ▁remembered ▁the ▁anniversary ▁I ▁forgot ▁. [SEP]


# Continue Training LORA

## Step 16: Load and Map

In [30]:
df = pd.read_csv("./data/final_finetune.csv")
df = preprocess_text_data(df, text_column="text")

df["label_id"] = df["label"].map(label2id)

dataset = Dataset.from_pandas(df[["text", "label_id"]])

## Step 17: Tokenize

In [31]:
tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset = tokenized_dataset.rename_column("label_id", "label")
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/123 [00:00<?, ? examples/s]

## Step 18: Training Args

In [32]:
training_args = TrainingArguments(
    output_dir="./lora_checkpoints/final_patch",
    per_device_train_batch_size=4,         # smaller batch → more granular LoRA updates
    gradient_accumulation_steps=2,         # effective batch size = 8
    num_train_epochs=6,                    # memorize-level training
    learning_rate=5e-5,                    # much better for LoRA tuning
    weight_decay=0.0,                      # no need — your examples are high-quality
    warmup_ratio=0.1,                      # optional, helps smooth out first few steps
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=False,
    remove_unused_columns=False,
    overwrite_output_dir=True,
    run_name="lora-final"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

C:\Users\SunAda\AppData\Local\Temp\ipykernel_27608\2295891324.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


## Step 18: Train

In [33]:
trainer.train()

Step,Training Loss
10,1.473000
20,1.440300
30,1.365800
40,1.218500
50,1.594500
60,1.406400
70,1.536600
80,1.292200
90,1.449200


TrainOutput(global_step=96, training_loss=1.4100165168444316, metrics={'train_runtime': 7.5275, 'train_samples_per_second': 98.041, 'train_steps_per_second': 12.753, 'total_flos': 6492342721680.0, 'train_loss': 1.4100165168444316, 'epoch': 6.0})

# Testing

In [34]:
## Step 6: Save LoRA adapter only
model.save_pretrained("lora_checkpoints/final_patch")
tokenizer.save_pretrained("lora_checkpoints/final_patch")
# Load base
base_model = AutoModelForSequenceClassification.from_pretrained("model/best_model_f")

# Apply LoRA adapter
model = PeftModel.from_pretrained(base_model, 
                                  "lora_checkpoints/final_patch", 
                                  label2id=label2id,
                                    id2label=id2label)
model.print_trainable_parameters()
# Sanity check: are LoRA adapters active?
print("Active adapters:", model.peft_config)

trainable params: 4,614 || all params: 185,021,196 || trainable%: 0.0025
Active adapters: {'default': LoraConfig(task_type='SEQ_CLS', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='./model/best_model_f', revision=None, inference_mode=True, r=16, target_modules={'query_proj', 'value_proj'}, exclude_modules=None, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=['classifier', 'score', 'classifier', 'score', 'classifier', 'score', 'classifier', 'score', 'classifier', 'score'], init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, use_qalora=False, qalora_group_size=16, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)}


In [35]:
final_test_sentences = [
    "She whispered 'I love you,' but it sounded like goodbye.",        # love
    "I tried to smile at the joke, yet each word felt like a dagger.", # anger
    "He stepped into the clearing expecting calm—and found a riot.",   # surprise
    "I told myself it was okay, but my legs refused to move.",        # fear
    "She danced around the room, but her tears fell like rain.",      # sadness
    "He held the gift in trembling hands, though his heart leapt.",   # joy
]


In [36]:
results = predict_emotion(final_test_sentences)
for text, label_id, conf in results:
    label_id = id2label[label_id]
    print(f"{text} → {label_id} (conf: {conf:.2f})")
    print()
explainer = SequenceClassificationExplainer(
    model=model,
    tokenizer=tokenizer
)

She whispered 'I love you,' but it sounded like goodbye. → love (conf: 0.73)

I tried to smile at the joke, yet each word felt like a dagger. → joy (conf: 0.29)

He stepped into the clearing expecting calm—and found a riot. → surprise (conf: 0.52)

I told myself it was okay, but my legs refused to move. → joy (conf: 0.55)

She danced around the room, but her tears fell like rain. → surprise (conf: 0.54)

He held the gift in trembling hands, though his heart leapt. → fear (conf: 0.81)



In [ ]:
word_attributions = explainer("He remembered the anniversary I forgot.")
explainer.visualize()  # or print(word_attributions)